<a href="https://colab.research.google.com/github/UmarKhattab09/Recommendation_System/blob/main/KNN_COLLABRATIVE_FILTERING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### K-NEAREST NEIGHBHOR for Collaborative Filtering

- How it Works: It checks for Similar users who have a similar buying pattern and provide recommendation.

In [31]:
import numpy as pd
import pandas as pd

In [32]:
column_names = ["user_id","item_id","rating","timestamp"]
df = pd.read_csv("u.data",sep="\t",names=column_names)

In [10]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [33]:
names = ["movie_id","movie_title","release_date","video_release_date","IMDB_URL","Unknown","Action","Adventure","Animation","Children's",
         "Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","SciFi","Thriller","War","Western"]
df2 = pd.read_csv("u.item",sep="|",encoding="ISO-8859-1",names =names)
df2.head()

,movie_id,movie_title,release_date,video_release_date,IMDB_URL,Unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [34]:
movie_title= df2[["movie_id","movie_title"]]
movie_title.rename(columns={"movie_id":"item_id"},inplace=True)
movie_title.head()

/tmp/ipython-input-2007401347.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_title.rename(columns={"movie_id":"item_id"},inplace=True)


,item_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [35]:
#Merging
df = pd.merge(df,movie_title,on="item_id")
df.head()

,user_id,item_id,rating,timestamp,movie_title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [37]:
ratings = df.groupby("movie_title")["rating"].mean().reset_index().rename(columns={"rating":"avg_rating"})
ratings.head()
ratings["rating_count"] = df.groupby("movie_title")["rating"].count().reset_index()["rating"]
ratings.head()

,movie_title,avg_rating,rating_count
0,'Til There Was You (1997),2.333333,9
1,1-900 (1994),2.600000,5
2,101 Dalmatians (1996),2.908257,109
3,12 Angry Men (1957),4.344000,125
4,187 (1997),3.024390,41


In [38]:
df = pd.merge(df,ratings,on="movie_title")
df.head()

,user_id,item_id,rating,timestamp,movie_title,avg_rating,rating_count
0,196,242,3,881250949,Kolya (1996),3.991453,117
1,186,302,3,891717742,L.A. Confidential (1997),4.161616,297
2,22,377,1,878887116,Heavyweights (1994),2.153846,13
3,244,51,2,880606923,Legends of the Fall (1994),3.456790,81
4,166,346,1,886397596,Jackie Brown (1997),3.642857,126


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       100000 non-null  int64  
 1   item_id       100000 non-null  int64  
 2   rating        100000 non-null  int64  
 3   timestamp     100000 non-null  int64  
 4   movie_title   100000 non-null  object 
 5   avg_rating    100000 non-null  float64
 6   rating_count  100000 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 5.3+ MB


In [40]:
df["rating_count"].describe()

,rating_count
count,100000.000000
mean,169.088380
std,122.187866
min,1.000000
25%,72.000000
50%,146.000000
75%,240.000000
max,583.000000


In [41]:
threshold=50
threshold_df = df[df["rating_count"]>=threshold]
threshold_df.head()

,user_id,item_id,rating,timestamp,movie_title,avg_rating,rating_count
0,196,242,3,881250949,Kolya (1996),3.991453,117
1,186,302,3,891717742,L.A. Confidential (1997),4.161616,297
3,244,51,2,880606923,Legends of the Fall (1994),3.456790,81
4,166,346,1,886397596,Jackie Brown (1997),3.642857,126
5,298,474,4,884182806,Dr. Strangelove or: How I Learned to Stop Worr...,4.252577,194


In [42]:
threshold_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84069 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       84069 non-null  int64  
 1   item_id       84069 non-null  int64  
 2   rating        84069 non-null  int64  
 3   timestamp     84069 non-null  int64  
 4   movie_title   84069 non-null  object 
 5   avg_rating    84069 non-null  float64
 6   rating_count  84069 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 5.1+ MB


In [29]:
# threshold_df[threshold_df["item_id"] ==1]

In [43]:
threshold_df.shape

(84069, 7)

In [46]:
movie_matrix= threshold_df.pivot_table(index="movie_title",columns="user_id",values="rating").fillna(0)
movie_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie_title,,,,,,,,,,,,,,,,,,,,,
101 Dalmatians (1996),2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),5.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2 Days in the Valley (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
"20,000 Leagues Under the Sea (1954)",3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),4.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0,0.0,5.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [51]:
from scipy.sparse import csr_matrix
movie_matrix_sparse = csr_matrix(movie_matrix.values)

In [52]:
movie_matrix_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 83774 stored elements and shape (605, 943)>

In [53]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(movie_matrix_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [63]:
import numpy as np
query_index = np.random.choice(movie_matrix.shape[0])

In [71]:
query_index
movie_matrix.index[query_index]

"Devil's Own, The (1997)"

When you use `movie_matrix.shape`, it returns a tuple representing the dimensions of your DataFrame. For example, if `movie_matrix` has 605 rows and 943 columns, `movie_matrix.shape` would return `(605, 943)`.

*   `movie_matrix.shape[0]` refers to the **number of rows** in your `movie_matrix`. In your case, this represents the total number of unique movies that meet your `threshold` criteria (movies with at least 50 ratings). The output from a previous cell `(605, 943)` indicates there are 605 movies.

The `np.random.choice()` function takes an integer `a` as its first argument and returns a random sample from `np.arange(a)`. So, `np.random.choice(movie_matrix.shape[0])` means it will randomly pick an integer between `0` and `movie_matrix.shape[0] - 1` (which is `0` to `604` in your case). This integer corresponds to a random index for one of your movies (rows) in the `movie_matrix`.

*   `movie_matrix.shape[1]` refers to the **number of columns** in your `movie_matrix`. This represents the total number of unique users who have rated movies that meet your `threshold` criteria. In your case, this is `943` users. If you were to use `np.random.choice(movie_matrix.shape[1])`, you would get a random index for a user, not a movie.

In [69]:
distances,indices = model_knn.kneighbors(movie_matrix.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [70]:
for i in range(0,len(distances.flatten())):
  if i ==0:
    print(f"Recommendation for {movie_matrix.index[query_index]}")
  else:
    print(f"{i}:{movie_matrix.index[indices.flatten()[i]]}")

Recommendation for Devil's Own, The (1997)
1:Air Force One (1997)
2:Conspiracy Theory (1997)
3:Murder at 1600 (1997)
4:Liar Liar (1997)
5:Saint, The (1997)


In [72]:
from sklearn.decomposition import PCA
print("PCA imported successfully.")

PCA imported successfully.


In [73]:
pca = PCA(n_components=3)
movie_pca_components = pca.fit_transform(movie_matrix)
movie_pca = pd.DataFrame(data=movie_pca_components, columns=['PC1', 'PC2', 'PC3'], index=movie_matrix.index)
print("PCA applied and new DataFrame 'movie_pca' created.")
movie_pca.head()

PCA applied and new DataFrame 'movie_pca' created.


,PC1,PC2,PC3
movie_title,,,
101 Dalmatians (1996),-8.858413,-4.319623,-6.299288
12 Angry Men (1957),4.996132,12.051617,6.380692
2 Days in the Valley (1996),-9.358661,-6.967567,-0.177037
"20,000 Leagues Under the Sea (1954)",-5.743903,7.143085,-5.454236
2001: A Space Odyssey (1968),28.674428,15.549907,7.545074


In [76]:
import plotly.express as px


In [75]:
fig = px.scatter_3d(movie_pca, x='PC1', y='PC2', z='PC3',
                    hover_name=movie_pca.index,
                    title='3D PCA of Movies by User Ratings')
fig.show()
print("3D scatter plot generated successfully.")

3D scatter plot generated successfully.


## Summary:

### Data Analysis Key Findings
*   Principal Component Analysis (PCA) was successfully applied to the `movie_matrix`, reducing its high-dimensional representation to three principal components (PC1, PC2, PC3). This transformation creates a 3D numerical representation of movies based on user ratings.
*   An interactive 3D scatter plot was generated using these three principal components. Each point in the plot represents a movie, and its title is displayed upon hovering, enabling visual exploration of movie relationships in the reduced dimension space.

### Insights or Next Steps
*   The 3D PCA visualization provides a compact and explorable representation of movies, which can reveal inherent structures or groupings within the dataset.
*   The next step involves a detailed visual inspection of the interactive 3D scatter plot to identify any noticeable clusters, patterns, or outliers among the movies. This observation could lead to insights into movie genres, user preference groups, or other latent characteristics.
